In [ ]:
import os
import datetime
import sys
from pathlib import Path

curdir = Path(os.getcwd())
sys.path.append(str(curdir.parent.absolute()))

import json
import logging
from src.utils.data import read_pickle
from src.utils.losses import contrastive_loss
from src.data.collators import collate_variable_sequence_length
from src.data.datasets import ProteinDataset
from src.models.ProTCL import ProTCL
import torch
import wandb


# Data paths
TRAIN_DATA_PATH = '/home/ncorley/protein/ProteinFunctions/data/swissprot/proteinfer_splits/random/train_GO.fasta'
VAL_DATA_PATH = '/home/ncorley/protein/ProteinFunctions/data/swissprot/proteinfer_splits/random/dev_GO.fasta'
TEST_DATA_PATH = '/home/ncorley/protein/ProteinFunctions/data/swissprot/proteinfer_splits/random/test_GO.fasta'
AMINO_ACID_VOCAB_PATH = '/home/ncorley/protein/ProteinFunctions/data/vocabularies/amino_acid_vocab.json'
GO_LABEL_VOCAB_PATH = '/home/ncorley/protein/ProteinFunctions/data/vocabularies/GO_label_vocab.json'

# Embedding paths
LABEL_EMBEDDING_PATH = "/home/ncorley/protein/ProteinFunctions/data/embeddings/label_embeddings.pk1"
SEQUENCE_EMBEDDING_PATH = "/home/ncorley/protein/ProteinFunctions/data/embeddings/sequence_embeddings.pk1"

# Load datasets
train_dataset, val_dataset, test_dataset = ProteinDataset\
    .create_multiple_datasets(data_paths=[TRAIN_DATA_PATH, VAL_DATA_PATH, TEST_DATA_PATH],
                              sequence_vocabulary_path=AMINO_ACID_VOCAB_PATH)

# Create label voculabary by merging sets from train_loader, val_loader, and test_loader
master_label_vocabulary = list(set(train_dataset.label_vocabulary) | set(val_dataset.label_vocabulary) | set(test_dataset.label_vocabulary))

# Save master_label_vocabulary to JSON file
with open('/home/ncorley/protein/ProteinFunctions/data/vocabularies/GO_label_vocab.json', 'w') as f:
    json.dump(master_label_vocabulary, f)

In [11]:
import os
import datetime
import sys
from pathlib import Path
import torch

curdir = Path(os.getcwd())
sys.path.append(str(curdir.parent.absolute()))

# Import read_pickle
from src.utils.data import read_pickle

# Load /home/ncorley/protein/ProteinFunctions/data/embeddings/proteinfer_sequence_id_map.pkl
sequence_id_map = read_pickle('/home/ncorley/protein/ProteinFunctions/data/embeddings/proteinfer_sequence_id_map.pkl')

# Print the shape and column names
print(len(sequence_id_map))

# Print the first 10 keys
print(list(sequence_id_map.keys())[:10])

# Check if the keys are unique
print(len(sequence_id_map) == len(set(sequence_id_map.keys())))

# Load sequence embeddings 
sequence_embeddings = read_pickle('/home/ncorley/protein/ProteinFunctions/data/embeddings/proteinfer_sequence_embeddings.pkl')
print(len(sequence_embeddings))

# Load old embeddings from /home/ncorley/protein/ProteinFunctions/data/embeddings/sequence_embeddings.pth with pytorch
old_sequence_embeddings = torch.load('/home/ncorley/protein/ProteinFunctions/data/embeddings/sequence_embeddings.pth')

# Print number of old embeddings
print(len(old_sequence_embeddings))

522607
['A0A023GPI8', 'A0A023IWD9', 'A0A023IWE0', 'A0A023IWE1', 'A0A023IWE2', 'A0A023IWE3', 'A0A023IWG1', 'A0A023IWG2', 'A0A023IWG3', 'A0A023IWG4']
True
522607
522607
439540


# Create new map from label ID to tokens

In [32]:
from pathlib import Path
import os
import sys
import gzip
import wget

curdir = Path(os.getcwd())
sys.path.append(str(curdir.parent.absolute()))


In [33]:
# Load vocab from /home/ncorley/protein/ProteinFunctions/data/vocabularies/GO_label_vocab.json
import json
with open('/home/ncorley/protein/ProteinFunctions/data/vocabularies/GO_label_vocab.json', 'r') as f:
    label_vocab = json.load(f)

# Print the first 10 labels
print(label_vocab[:10])

['GO:0000001', 'GO:0000002', 'GO:0000003', 'GO:0000006', 'GO:0000007', 'GO:0000009', 'GO:0000010', 'GO:0000011', 'GO:0000012', 'GO:0000014']


In [57]:
# Load annotations from /home/ncorley/protein/ProteinFunctions/data/annotations/go_annotations_2019_07_01.pkl
from src.utils.data import read_pickle
annotations = read_pickle('/home/ncorley/protein/ProteinFunctions/data/annotations/go_annotations_2019_07_01.pkl')

In [72]:
# Filter the annotations df to be only the labels in label_vocab. In annotations, the go id is the index
annotations = annotations[annotations.index.isin(label_vocab)]

In [73]:
# Load PubMedBERT model
from src.utils.models import load_PubMedBERT
tokenizer, model = load_PubMedBERT()

Some weights of the model checkpoint at microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [61]:
# Load PubMedBERT tokenizer
inputs = tokenizer(["Test text This is what it is like to live! I will run forever!", "BLAH"], return_tensors="pt",
                       truncation=True, padding=True, max_length=512)
print(inputs)
print(type(inputs))

{'input_ids': tensor([[    2,  2648,  5316,  2052,  1977,  4663,  2176,  1977,  3189,  1942,
          6922,     5,    51,  2832,  4747,  9023,  2004,     5,     3],
        [    2, 18208,  1023,     3,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])}
<class 'transformers.tokenization_utils_base.BatchEncoding'>


In [62]:
annotations.loc[:, 'tokenized_label_text'] = annotations['label'].apply(lambda x: tokenizer(x, return_tensors="pt", truncation=True, padding=True, max_length=512))

In [81]:
annotations.columns

Index(['label', 'tokenized_label_text', 'input_ids', 'attention_mask',
       'token_type_ids'],
      dtype='object')

In [74]:
# Tokenize all labels in the dataframe in a batched manner
tokenized_outputs = tokenizer(annotations['label'].tolist(), return_tensors="pt", truncation=True, padding=True, max_length=512)


275

In [ ]:
# Assign the tokenized outputs to the dataframe
annotations['input_ids'] = tokenized_outputs['input_ids']
annotations['attention_mask'] = tokenized_outputs['attention_mask']
if 'token_type_ids' in tokenized_outputs:
    annotations['token_type_ids'] = tokenized_outputs['token_type_ids']

In [66]:
import torch

# Extract input_ids and attention_mask as tensors and create new columns
annotations['input_ids'] = annotations['tokenized_label_text'].apply(lambda x: x['input_ids'].clone().detach())
annotations['attention_mask'] = annotations['tokenized_label_text'].apply(lambda x: x['attention_mask'].clone().detach())
annotations['token_type_ids'] = annotations['tokenized_label_text'].apply(lambda x: x.get('token_type_ids', torch.zeros_like(x['input_ids'])).clone().detach())

# # Drop the tokenized_label_text column
tokenized_annotations = annotations.drop(columns=['label', 'tokenized_label_text'])

# Save tokenized_annotations
tokenized_annotations.to_pickle('/home/ncorley/protein/ProteinFunctions/data/annotations/tokenized_go_annotations_2019_07_01.pkl')


In [68]:
import pandas as pd

# Load the pickled DataFrame
df = pd.read_pickle('/home/ncorley/protein/ProteinFunctions/data/annotations/tokenized_go_annotations_2019_07_01.pkl')

# Create dictionaries for each column using the DataFrame's index as the key

input_ids_dict = df['input_ids'].to_dict()
attention_mask_dict = df['attention_mask'].to_dict()
token_type_ids_dict = df['token_type_ids'].to_dict()

32102


In [71]:
print(len(token_type_ids_dict))

32102
